# Phase II 
Simran Bhamra, Albert Yeh 

# Introduction 
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; The goal of this project is to apply the data cleaning concepts and practices we learned during class to crime data from the city of Denver, Colorado. The Denver City crime data is part of an open data catalog provided by the city government and collected by the Denver Police Department. The data can be found at this location: https://www.denvergov.org/opendata/dataset/city-and-county-of-denver-crime. For this project we will be using two of the tables provided, the first is the crime table which contains all criminal offenses in the city and county of Denver from January 2017- June 2022. The second is the offense code table, this can be used to cross-reference the offense code given in table 1 to find the actual crime name according to the National Incident-Based Reporting System.


In [2]:
import numpy as np
import pandas as pd
import datetime as dt

# Methods 
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; The first step in our data cleaning workflow is to import both our data sets and begin to change the data to the correct types. We do this with the crimes table first using some of the data we collected during our data exploration in phase I. 

In [3]:
#import datasets
crimes = pd.read_csv("crime.csv", encoding='latin-1', engine='python')
offense_codes = pd.read_csv("offense_codes.csv", encoding='latin-1', engine='python')

In [4]:
crimes.head()

,incident_id,offense_id,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,REPORTED_DATE,INCIDENT_ADDRESS,GEO_X,GEO_Y,GEO_LON,GEO_LAT,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID,IS_CRIME,IS_TRAFFIC,VICTIM_COUNT
0,20226000193,20226000193299900,2999,0,criminal-mischief-other,public-disorder,1/4/2022 11:30:00 AM,1/4/2022 12:00:00 PM,1/4/2022 8:36:00 PM,128 S CANOSA CT,3135366.0,1685410.0,-105.018825,39.714268,4.0,411.0,valverde,1,0,1
1,20223319,20223319299900,2999,0,criminal-mischief-other,public-disorder,1/3/2022 6:45:00 AM,NaN,1/3/2022 11:01:00 AM,650 15TH ST,3142454.0,1696151.0,-104.993418,39.743649,6.0,611.0,cbd,1,0,1
2,20223093,20223093299900,2999,0,criminal-mischief-other,public-disorder,1/3/2022 1:00:00 AM,NaN,1/3/2022 6:11:00 AM,919 E COLFAX AVE,3147484.0,1694898.0,-104.975557,39.740130,6.0,621.0,north-capitol-hill,1,0,1
3,20224000,20224000299900,2999,0,criminal-mischief-other,public-disorder,1/3/2022 7:47:00 PM,NaN,1/3/2022 9:12:00 PM,2345 W ALAMEDA AVE,3136478.0,1684414.0,-105.014892,39.711518,4.0,411.0,valverde,1,0,1
4,20223956,20223956299900,2999,0,criminal-mischief-other,public-disorder,1/3/2022 5:06:00 PM,NaN,1/3/2022 8:31:00 PM,7800 E SMITH RD,3169237.0,1705800.0,-104.897950,39.769688,5.0,512.0,central-park,1,0,1


## Step 1 
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; First the FIRST_OCCURRENCE_DATE and REPORTED_DATE columns are updated to the correct type as planned in phase I. Initially both are of type object, we chose to convert them to pandas datetime objects and ISO format as both could be useful. Further we can see that 376812 values of FIRST_OCCURRENCE_DATE and REPORTED_DATE are changed here. Once the type has been changed FIRST_OCCURRENCE_DATE with datetime formatting is saved as first_occurrence_date_dt and ISO formatting is saved as first_occurrence_date_isoformat and added to the crimes dataframe as new columns. Similarly with REPORTED_DATE the columns are saved as reported_date_dt and reported_date_isoformat respectfully.

In [5]:
crimes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 376812 entries, 0 to 376811
Data columns (total 20 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   incident_id             376812 non-null  int64  
 1   offense_id              376812 non-null  int64  
 2   OFFENSE_CODE            376812 non-null  int64  
 3   OFFENSE_CODE_EXTENSION  376812 non-null  int64  
 4   OFFENSE_TYPE_ID         376812 non-null  object 
 5   OFFENSE_CATEGORY_ID     376812 non-null  object 
 6   FIRST_OCCURRENCE_DATE   376812 non-null  object 
 7   LAST_OCCURRENCE_DATE    192816 non-null  object 
 8   REPORTED_DATE           376812 non-null  object 
 9   INCIDENT_ADDRESS        371545 non-null  object 
 10  GEO_X                   371545 non-null  float64
 11  GEO_Y                   371545 non-null  float64
 12  GEO_LON                 370963 non-null  float64
 13  GEO_LAT                 370963 non-null  float64
 14  DISTRICT_ID         

In [6]:
sum(crimes['FIRST_OCCURRENCE_DATE'].str.endswith("M"))

376812

In [7]:
sum(crimes['REPORTED_DATE'].str.endswith("M"))

376812

In [8]:
# date type conversions for FIRST_OCCURRENCE_DATE
crimes['first_occurrence_date_dt'] = pd.to_datetime(crimes['FIRST_OCCURRENCE_DATE'], format="%m/%d/%Y %I:%M:%S %p")
crimes['first_occurrence_date_isoformat'] = crimes['first_occurrence_date_dt'].apply(dt.datetime.isoformat)

In [9]:
# date type conversions for REPORTED_DATE
crimes['reported_date_dt'] = pd.to_datetime(crimes['REPORTED_DATE'], format="%m/%d/%Y %I:%M:%S %p")
crimes['reported_date_isoformat'] = crimes['reported_date_dt'].apply(dt.datetime.isoformat)

## Step 2
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Using .strip() the whitespace before and after the following columns: OFFENSE_TYPE_ID, OFFENSE_CATEGORY_ID, INCIDENT_ADDRESS, NEIGHBORHOOD_ID. These are columns that include strings which in pandas dataframes are converted to objects. Before performing the trim we see below that the columns don't have any leading or trailing whitespace, which we did not expect. We still run the trim function here and check for whitespaces again to confirm there are no whitespaces present.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; In addition, the special characters are removed from the INCIDENT_ADDRESS column. The columns are also renamed to OFFENSE_TYPE_ID_trim, OFFENSE_CATEGORY_ID_trim, INCIDENT_ADDRESS_trim, and NEIGHBORHOOD_ID_trim. Thus far all of the steps have been planned for and executed from phase I, however the next step listed in phase I : "adjust address column name or add city, state,and zip to make a full address" is skipped. This is because, in order to make a full address with zip and city we would need to query an outside API which would go beyond the scope of this project. Instead this column is renamed to INCIDENT_STREET which more accuratly describes the data. 

In [10]:
# checking for leading and trailing white spaces 
crimes['OFFENSE_TYPE_ID'].str.startswith(" ").sum() + crimes['OFFENSE_TYPE_ID'].str.endswith(" ").sum()

0

In [11]:
# checking for leading and trailing white spaces 
crimes['OFFENSE_CATEGORY_ID'].str.startswith(" ").sum() + crimes['OFFENSE_CATEGORY_ID'].str.endswith(" ").sum()

0

In [12]:
# checking for leading and trailing white spaces 
crimes['INCIDENT_ADDRESS'].str.startswith(" ").sum() + crimes['INCIDENT_ADDRESS'].str.endswith(" ").sum()

0

In [13]:
# checking for leading and trailing white spaces 
crimes['NEIGHBORHOOD_ID'].str.startswith(" ").sum() + crimes['NEIGHBORHOOD_ID'].str.endswith(" ").sum()

0

In [14]:
# running .trim() and adding new columns with results 
crimes['OFFENSE_TYPE_ID_trim'] = crimes.OFFENSE_TYPE_ID.str.strip()
crimes['OFFENSE_CATEGORY_ID_trim'] = crimes.OFFENSE_CATEGORY_ID.str.strip()
crimes['INCIDENT_ADDRESS_trim'] = crimes.INCIDENT_ADDRESS.str.strip()
crimes['NEIGHBORHOOD_ID_trim'] = crimes.NEIGHBORHOOD_ID.str.strip()

In [15]:
# double checking whitespace after running .trim()
crimes['OFFENSE_TYPE_ID_trim'].str.startswith(" ").sum() + crimes['OFFENSE_TYPE_ID_trim'].str.endswith(" ").sum()

0

In [16]:
# double checking whitespace after running .trim()
crimes['OFFENSE_CATEGORY_ID_trim'].str.startswith(" ").sum() + crimes['OFFENSE_CATEGORY_ID_trim'].str.endswith(" ").sum()

0

In [17]:
# double checking whitespace after running .trim()
crimes['INCIDENT_ADDRESS_trim'].str.startswith(" ").sum() + crimes['INCIDENT_ADDRESS_trim'].str.endswith(" ").sum()

0

In [18]:
# double checking whitespace after running .trim()
crimes['NEIGHBORHOOD_ID_trim'].str.startswith(" ").sum() + crimes['NEIGHBORHOOD_ID_trim'].str.endswith(" ").sum()

0

In [19]:
# checking how many incident addresses contain special characters 
crimes['INCIDENT_ADDRESS_trim'].str.contains(r'[^\w\s]|_', regex=True).sum()

32305

In [20]:
# remove special characters from incident_address_trim 
crimes['INCIDENT_ADDRESS_trim'] = crimes['INCIDENT_ADDRESS_trim'].replace(r'[^\w\s]|_', '', regex=True)

In [21]:
# checking how many incident addresses contain special characters after removing with regex
crimes['INCIDENT_ADDRESS_trim'].str.contains(r'[^\w\s]|_', regex=True).sum()

0

In [22]:
# renaming INCIDENT_ADDRESS_trim
crimes.rename(columns = {'INCIDENT_ADDRESS_trim':'INCIDENT_STREET'}, inplace = True)

## Step 3
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Initally mentioned later in the phase I method a few columns are dropped in this step. Columns are dropped here as well as when joining the tables as planned because many columns are now dupicated version we have made in conversions, some columns are not needed, and it makes the dataframe easier to work with as there are less columns to keep track of.  

In [23]:
# listing all the columns we currently have 
list(crimes)

['incident_id',
 'offense_id',
 'OFFENSE_CODE',
 'OFFENSE_CODE_EXTENSION',
 'OFFENSE_TYPE_ID',
 'OFFENSE_CATEGORY_ID',
 'FIRST_OCCURRENCE_DATE',
 'LAST_OCCURRENCE_DATE',
 'REPORTED_DATE',
 'INCIDENT_ADDRESS',
 'GEO_X',
 'GEO_Y',
 'GEO_LON',
 'GEO_LAT',
 'DISTRICT_ID',
 'PRECINCT_ID',
 'NEIGHBORHOOD_ID',
 'IS_CRIME',
 'IS_TRAFFIC',
 'VICTIM_COUNT',
 'first_occurrence_date_dt',
 'first_occurrence_date_isoformat',
 'reported_date_dt',
 'reported_date_isoformat',
 'OFFENSE_TYPE_ID_trim',
 'OFFENSE_CATEGORY_ID_trim',
 'INCIDENT_STREET',
 'NEIGHBORHOOD_ID_trim']

### Dropped columns

| Column Name      | Reason Dropped |
| :---       |    :---    |  
| LAST_OCCURRENCE_DATE      | large amount of missing data            | 
| FIRST_OCCURRENCE_DATE     | replaced with formatted version of date | 
| REPORTED_DATE             | replaced with formatted version of date |
| OFFENSE_TYPE_ID           | we will be using the offense data from the offense table when joined to provide more accurate description so this is not needed                            |
|OFFENSE_CATEGORY_ID        | we will be using the offense data from the offense table when joined to provide more accurate description so this is not needed                            |
|INCIDENT_ADDRESS           | trimed, special characters removed and renamed to INCIDENT_STREET for data accuracy |
|NEIGHBORHOOD_ID            | replaced with NEIGHBORHOOD_ID_trim      |

In [24]:
# missing data
crimes['LAST_OCCURRENCE_DATE'].isna().sum()

183996

In [25]:
# dropping columns
crimes = crimes.drop(columns=['LAST_OCCURRENCE_DATE','FIRST_OCCURRENCE_DATE','REPORTED_DATE','OFFENSE_TYPE_ID','OFFENSE_CATEGORY_ID','INCIDENT_ADDRESS','NEIGHBORHOOD_ID'])


In [26]:
# columns after drop 
list(crimes)

['incident_id',
 'offense_id',
 'OFFENSE_CODE',
 'OFFENSE_CODE_EXTENSION',
 'GEO_X',
 'GEO_Y',
 'GEO_LON',
 'GEO_LAT',
 'DISTRICT_ID',
 'PRECINCT_ID',
 'IS_CRIME',
 'IS_TRAFFIC',
 'VICTIM_COUNT',
 'first_occurrence_date_dt',
 'first_occurrence_date_isoformat',
 'reported_date_dt',
 'reported_date_isoformat',
 'OFFENSE_TYPE_ID_trim',
 'OFFENSE_CATEGORY_ID_trim',
 'INCIDENT_STREET',
 'NEIGHBORHOOD_ID_trim']

## Step 4 
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Next NEIGHBORHOOD_ID_trim must be cleaned. In phase I one of the steps were to consolidate neighborhood ID column and remove garbage values. This ended up requireing more work than expected because of all the missing values. Since this is an important column we calculate haversine distance / great circle distance between each incident location and all other average neighborhood locations. Before this we ensure that columns are named correctly, check the uniqueness, and calculate the NA amount. 

In [27]:
crimes['NEIGHBORHOOD_ID_trim'].unique()

array(['valverde', 'cbd', 'north-capitol-hill', 'central-park',
       'cory-merrill', 'capitol-hill', 'belcaro', 'lincoln-park',
       'villa-park', 'cheesman-park', 'virginia-village', 'highland',
       'windsor', 'marston', 'five-points', 'west-colfax', 'sun-valley',
       'hampden-south', 'hampden', 'dia', 'university', 'cherry-creek',
       'gateway-green-valley-ranch', 'east-colfax', 'west-highland',
       'mar-lee', 'elyria-swansea', 'union-station', 'university-park',
       'jefferson-park', 'washington-park', 'lowry-field',
       'university-hills', 'goldsmith', 'baker', 'city-park-west',
       'civic-center', 'ruby-hill', 'northeast-park-hill', 'barnum-west',
       'montbello', 'wellshire', 'speer', 'fort-logan', 'bear-valley',
       'overland', 'kennedy', 'harvey-park-south', 'sunnyside',
       'athmar-park', 'hilltop', 'harvey-park', 'clayton', 'westwood',
       'globeville', 'regis', 'hale', 'washington-virginia-vale', 'cole',
       'south-park-hill', 'sloan-l

In [28]:
len(crimes['NEIGHBORHOOD_ID_trim'].unique())

79

In [29]:
#covert neighborhood_ID_trim with name cbd to central business district for clarity
crimes['NEIGHBORHOOD_ID_trim'] = crimes['NEIGHBORHOOD_ID_trim'].replace({'cbd':'central-business-district'})

In [30]:
crimes['NEIGHBORHOOD_ID_trim'].unique()

array(['valverde', 'central-business-district', 'north-capitol-hill',
       'central-park', 'cory-merrill', 'capitol-hill', 'belcaro',
       'lincoln-park', 'villa-park', 'cheesman-park', 'virginia-village',
       'highland', 'windsor', 'marston', 'five-points', 'west-colfax',
       'sun-valley', 'hampden-south', 'hampden', 'dia', 'university',
       'cherry-creek', 'gateway-green-valley-ranch', 'east-colfax',
       'west-highland', 'mar-lee', 'elyria-swansea', 'union-station',
       'university-park', 'jefferson-park', 'washington-park',
       'lowry-field', 'university-hills', 'goldsmith', 'baker',
       'city-park-west', 'civic-center', 'ruby-hill',
       'northeast-park-hill', 'barnum-west', 'montbello', 'wellshire',
       'speer', 'fort-logan', 'bear-valley', 'overland', 'kennedy',
       'harvey-park-south', 'sunnyside', 'athmar-park', 'hilltop',
       'harvey-park', 'clayton', 'westwood', 'globeville', 'regis',
       'hale', 'washington-virginia-vale', 'cole', 'sout

In [31]:
len(crimes['NEIGHBORHOOD_ID_trim'].unique())

79

In [32]:
# print all rows with neighbourhood_ID_trim with na
crimes[crimes['NEIGHBORHOOD_ID_trim'].isna()]

,incident_id,offense_id,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,GEO_X,GEO_Y,GEO_LON,GEO_LAT,DISTRICT_ID,PRECINCT_ID,...,IS_TRAFFIC,VICTIM_COUNT,first_occurrence_date_dt,first_occurrence_date_isoformat,reported_date_dt,reported_date_isoformat,OFFENSE_TYPE_ID_trim,OFFENSE_CATEGORY_ID_trim,INCIDENT_STREET,NEIGHBORHOOD_ID_trim
359405,2022307631,2022307631299900,2999,0,3147195.0,1695029.0,NaN,NaN,NaN,NaN,...,0,1,2022-06-17 19:33:00,2022-06-17T19:33:00,2022-06-17 20:06:00,2022-06-17T20:06:00,criminal-mischief-other,public-disorder,857 E COLFAX AVE,NaN
359409,2022308867,2022308867299900,2999,0,3174073.0,1689429.0,NaN,NaN,NaN,NaN,...,0,1,2022-06-18 12:44:00,2022-06-18T12:44:00,2022-06-18 14:34:00,2022-06-18T14:34:00,criminal-mischief-other,public-disorder,9100 E LOWRY BLVD,NaN
359410,2022305932,2022305932299900,2999,0,3142958.0,1672196.0,NaN,NaN,NaN,NaN,...,0,1,2022-06-16 23:35:00,2022-06-16T23:35:00,2022-06-16 23:35:00,2022-06-16T23:35:00,criminal-mischief-other,public-disorder,2140 S DELAWARE ST,NaN
359412,2022310449,2022310449299900,2999,0,3184346.0,1712762.0,NaN,NaN,NaN,NaN,...,0,1,2022-06-19 09:25:00,2022-06-19T09:25:00,2022-06-19 11:17:00,2022-06-19T11:17:00,criminal-mischief-other,public-disorder,5063 N SCRANTON ST,NaN
359413,20226011416,20226011416299900,2999,0,3209045.0,1711845.0,NaN,NaN,NaN,NaN,...,0,1,2022-06-18 20:00:00,2022-06-18T20:00:00,2022-06-19 18:48:00,2022-06-19T18:48:00,criminal-mischief-other,public-disorder,4853 N HALIFAX CT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360347,20226011252,20226011252260900,2609,0,3130968.0,1686400.0,NaN,NaN,NaN,NaN,...,0,1,2022-06-11 12:45:00,2022-06-11T12:45:00,2022-06-16 10:50:00,2022-06-16T10:50:00,fraud-by-use-of-computer,white-collar-crime,20 N LOWELL BLVD,NaN
360348,2022300759,2022300759260900,2609,0,3140838.0,1694464.0,NaN,NaN,NaN,NaN,...,0,1,2022-05-31 00:00:00,2022-05-31T00:00:00,2022-06-14 11:00:00,2022-06-14T11:00:00,fraud-by-use-of-computer,white-collar-crime,901 W 14TH AVE,NaN
360349,2022306766,2022306766260900,2609,0,3163327.0,1709344.0,NaN,NaN,NaN,NaN,...,0,1,2022-04-28 08:00:00,2022-04-28T08:00:00,2022-06-17 13:27:00,2022-06-17T13:27:00,fraud-by-use-of-computer,white-collar-crime,5885 E STAPLETON N DR,NaN
360351,20228026207,20228026207269903,2699,3,3223441.0,1728516.0,NaN,NaN,NaN,NaN,...,0,1,2022-06-18 06:07:00,2022-06-18T06:07:00,2022-06-18 07:32:00,2022-06-18T07:32:00,theft-of-services,larceny,24300 E 75TH AVE,NaN


In [33]:
# get avg latlon of neighborhood. Not geodesic, but sufficient for approximation.

avg_neighborhood_locations = crimes.groupby(['NEIGHBORHOOD_ID_trim'],as_index=False)['GEO_X','GEO_Y'].mean()

# calculate haversine distance / great circle distance between each incident location and all other average neighborhood locations
# https://en.wikipedia.org/wiki/Haversine_formula
# credit: https://stackoverflow.com/questions/29545704/fast-haversine-approximation-python-pandas/29546836#29546836

def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km


dist_df = crimes[crimes['NEIGHBORHOOD_ID_trim'].isna()][['incident_id','GEO_X','GEO_Y']].merge(avg_neighborhood_locations,how='cross')

dist_df['dist_km'] = haversine_np(dist_df['GEO_Y_x'],dist_df['GEO_X_x'],dist_df['GEO_Y_y'],dist_df['GEO_X_x'])

# get nearest neighborhood for each incident
nearest_neighborhoods = dist_df.sort_values(['dist_km'],ascending=True).groupby('incident_id',as_index=False).first().rename(columns={'NEIGHBORHOOD_ID_trim':'nearest_neighborhood','dist_km':'nearest_neighborhood_dist_km'})[['incident_id','nearest_neighborhood','nearest_neighborhood_dist_km']]

# merge back in to crimes dataset
crimes = crimes.merge(nearest_neighborhoods,on='incident_id',how='left')

crimes['nearest_neighborhood'] = np.where(crimes['nearest_neighborhood'].isna(),crimes['NEIGHBORHOOD_ID_trim'],crimes['nearest_neighborhood'])

# fill in known neighborhoods w/ a dist of 0
crimes['nearest_neighborhood_dist_km'] = np.where(crimes['nearest_neighborhood_dist_km'].isna(),0,crimes['nearest_neighborhood_dist_km'])

/Users/simranbhamra/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [34]:
crimes[crimes['nearest_neighborhood_dist_km']>0]

,incident_id,offense_id,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,GEO_X,GEO_Y,GEO_LON,GEO_LAT,DISTRICT_ID,PRECINCT_ID,...,first_occurrence_date_dt,first_occurrence_date_isoformat,reported_date_dt,reported_date_isoformat,OFFENSE_TYPE_ID_trim,OFFENSE_CATEGORY_ID_trim,INCIDENT_STREET,NEIGHBORHOOD_ID_trim,nearest_neighborhood,nearest_neighborhood_dist_km
359405,2022307631,2022307631299900,2999,0,3147195.0,1695029.0,NaN,NaN,NaN,NaN,...,2022-06-17 19:33:00,2022-06-17T19:33:00,2022-06-17 20:06:00,2022-06-17T20:06:00,criminal-mischief-other,public-disorder,857 E COLFAX AVE,NaN,hampden-south,47.158290
359409,2022308867,2022308867299900,2999,0,3174073.0,1689429.0,NaN,NaN,NaN,NaN,...,2022-06-18 12:44:00,2022-06-18T12:44:00,2022-06-18 14:34:00,2022-06-18T14:34:00,criminal-mischief-other,public-disorder,9100 E LOWRY BLVD,NaN,sloan-lake,73.254474
359410,2022305932,2022305932299900,2999,0,3142958.0,1672196.0,NaN,NaN,NaN,NaN,...,2022-06-16 23:35:00,2022-06-16T23:35:00,2022-06-16 23:35:00,2022-06-16T23:35:00,criminal-mischief-other,public-disorder,2140 S DELAWARE ST,NaN,northeast-park-hill,157.756212
359412,2022310449,2022310449299900,2999,0,3184346.0,1712762.0,NaN,NaN,NaN,NaN,...,2022-06-19 09:25:00,2022-06-19T09:25:00,2022-06-19 11:17:00,2022-06-19T11:17:00,criminal-mischief-other,public-disorder,5063 N SCRANTON ST,NaN,cory-merrill,269.985030
359413,20226011416,20226011416299900,2999,0,3209045.0,1711845.0,NaN,NaN,NaN,NaN,...,2022-06-18 20:00:00,2022-06-18T20:00:00,2022-06-19 18:48:00,2022-06-19T18:48:00,criminal-mischief-other,public-disorder,4853 N HALIFAX CT,NaN,university,365.588283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360347,20226011252,20226011252260900,2609,0,3130968.0,1686400.0,NaN,NaN,NaN,NaN,...,2022-06-11 12:45:00,2022-06-11T12:45:00,2022-06-16 10:50:00,2022-06-16T10:50:00,fraud-by-use-of-computer,white-collar-crime,20 N LOWELL BLVD,NaN,valverde,198.819651
360348,2022300759,2022300759260900,2609,0,3140838.0,1694464.0,NaN,NaN,NaN,NaN,...,2022-05-31 00:00:00,2022-05-31T00:00:00,2022-06-14 11:00:00,2022-06-14T11:00:00,fraud-by-use-of-computer,white-collar-crime,901 W 14TH AVE,NaN,sloan-lake,426.267842
360349,2022306766,2022306766260900,2609,0,3163327.0,1709344.0,NaN,NaN,NaN,NaN,...,2022-04-28 08:00:00,2022-04-28T08:00:00,2022-06-17 13:27:00,2022-06-17T13:27:00,fraud-by-use-of-computer,white-collar-crime,5885 E STAPLETON N DR,NaN,globeville,162.040917
360351,20228026207,20228026207269903,2699,3,3223441.0,1728516.0,NaN,NaN,NaN,NaN,...,2022-06-18 06:07:00,2022-06-18T06:07:00,2022-06-18 07:32:00,2022-06-18T07:32:00,theft-of-services,larceny,24300 E 75TH AVE,NaN,valverde,147.331810


## Step 5 
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; With sufficiently cleaned crimes data the offense data frame can now be joined, note we did not need to apply any cleaning to the offense data as it came already cleaned. The tables are joined on the OFFENSE_CODE and OFFENSE_CODE_EXTENSION using a left join into a new table called dv_crime_data. To ensure there are no duplicates when joining the data we check that IS_CRIME_x', 'IS_TRAFFIC_x', is equal to 'IS_CRIME_y','IS_TRAFFIC_y', the 'X' version coming from the crimes data and 'Y' version from offense codes.


In [35]:
# join table
dv_crime_data = crimes.merge(offense_codes, on=['OFFENSE_CODE','OFFENSE_CODE_EXTENSION'],how='left')

In [36]:
# check for duplicates
duplicate_list = ['IS_CRIME', 'IS_TRAFFIC']

for duplicate in duplicate_list:
    pd.testing.assert_series_equal(dv_crime_data[duplicate+'_x'],dv_crime_data[duplicate+'_y'],check_names=False)

## Step 6 
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; With the joined tables there are more redundant columns to drop. After dropping the columns the column names are standardized.

### Dropped columns

| Column Name | Reason Dropped |
| :---       |    :---    |  
| GEO_X      | GEO_LAT and GEO_LON will be used for location so this is not needed  | 
| GEO_Y      | GEO_LAT and GEO_LON will be used for location so this is not needed  | 
|DISTRICT_ID | not relevant to analytic goal | 
| nearest_neighborhood_dist_km | can be disregarded  after we have the nearest neighborhoods |
| OBJECTID | count from offense table |
| OFFENSE_TYPE_ID | the name will be used instead of the ID | 
| OFFENSE_CATEGORY_ID | the name will be used instead of the ID | 
| IS_CRIME_y | duplicate of IS_CRIME_x from crimes table|
| IS_TRAFFIC_y | duplicate of IS_TRAFFIC_x from crimes table|
|first_occurrence_date_dt| ISO date format will be used so datetime format can be dropped |
| reported_date_dt |ISO date format will be used so datetime format can be dropped |
| OFFENSE_TYPE_ID_trim |the name will be used instead of the ID |
| OFFENSE_CATEGORY_ID_trim |the name will be used instead of the ID |




In [37]:
columns_to_drop = [
    'GEO_X',
    'GEO_Y', 'DISTRICT_ID',
    'nearest_neighborhood_dist_km',
    'OBJECTID',
    'OFFENSE_TYPE_ID',
    'OFFENSE_CATEGORY_ID',
    'IS_CRIME_y',
    'IS_TRAFFIC_y',
    'first_occurrence_date_dt',
    'reported_date_dt',
    'OFFENSE_TYPE_ID_trim',
    'OFFENSE_CATEGORY_ID_trim'
]
dv_crime_data = dv_crime_data.drop(columns = columns_to_drop)

In [38]:
# current columns
list(dv_crime_data)


['incident_id',
 'offense_id',
 'OFFENSE_CODE',
 'OFFENSE_CODE_EXTENSION',
 'GEO_LON',
 'GEO_LAT',
 'PRECINCT_ID',
 'IS_CRIME_x',
 'IS_TRAFFIC_x',
 'VICTIM_COUNT',
 'first_occurrence_date_isoformat',
 'reported_date_isoformat',
 'INCIDENT_STREET',
 'NEIGHBORHOOD_ID_trim',
 'nearest_neighborhood',
 'OFFENSE_TYPE_NAME',
 'OFFENSE_CATEGORY_NAME']

In [39]:
# convert all column names to lowercase 
dv_crime_data.columns = [x.lower() for x in dv_crime_data.columns]

In [40]:
# lowercase columns
list(dv_crime_data)

['incident_id',
 'offense_id',
 'offense_code',
 'offense_code_extension',
 'geo_lon',
 'geo_lat',
 'precinct_id',
 'is_crime_x',
 'is_traffic_x',
 'victim_count',
 'first_occurrence_date_isoformat',
 'reported_date_isoformat',
 'incident_street',
 'neighborhood_id_trim',
 'nearest_neighborhood',
 'offense_type_name',
 'offense_category_name']

In [41]:
# columns renamed to give more accurate descriptions 
dv_crime_data.rename(columns = {
    'is_crime_x':'is_crime',
    'is_traffic_x':'is_traffic',
    'first_occurrence_date_isoformat':'first_occurrence_date',
    'reported_date_isoformat':'reported_date',
    'neighborhood_id_trim':'neighborhood_id'
}, inplace = True)

In [42]:
# final columns
list(dv_crime_data)


['incident_id',
 'offense_id',
 'offense_code',
 'offense_code_extension',
 'geo_lon',
 'geo_lat',
 'precinct_id',
 'is_crime',
 'is_traffic',
 'victim_count',
 'first_occurrence_date',
 'reported_date',
 'incident_street',
 'neighborhood_id',
 'nearest_neighborhood',
 'offense_type_name',
 'offense_category_name']

## Step 7 
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; As planned for in phase I integrity constraints are checked. First, we check that there are no primary key violations, meaning that there will be no duplicates. Here incident_id and offense_id are used as the primary key, since a single incident can have multiple offenses occur.


In [43]:
# checking primary key violations 
primary_key_violations = dv_crime_data[dv_crime_data.duplicated(subset=['incident_id','offense_id'], keep=False)]
primary_key_violations

,incident_id,offense_id,offense_code,offense_code_extension,geo_lon,geo_lat,precinct_id,is_crime,is_traffic,victim_count,first_occurrence_date,reported_date,incident_street,neighborhood_id,nearest_neighborhood,offense_type_name,offense_category_name


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; With no primary key violations, the next violation to check is not null. We do not need to check for unquie constraints as for the remainder of the columns values can be repeated, for examples crimes can happen on the same day, or in the same neighborhood. 

In [44]:
# check for columns with na 
rows_w_na = dv_crime_data[dv_crime_data.isna().any(axis=1)]
rows_w_na

,incident_id,offense_id,offense_code,offense_code_extension,geo_lon,geo_lat,precinct_id,is_crime,is_traffic,victim_count,first_occurrence_date,reported_date,incident_street,neighborhood_id,nearest_neighborhood,offense_type_name,offense_category_name
93,20222312,20222312360101,3601,1,NaN,NaN,311.0,1,0,1,2022-01-02T13:00:00,2022-01-02T15:00:00,NaN,speer,speer,Fondling of an adult,Sexual Assault
163,20226052,20226052110200,1102,0,NaN,NaN,221.0,1,0,1,2022-01-03T12:00:00,2022-01-05T03:53:00,NaN,northeast-park-hill,northeast-park-hill,Rape,Sexual Assault
164,20229009,20229009110900,1109,0,NaN,NaN,323.0,1,0,1,2022-01-06T20:30:00,2022-01-06T20:40:00,NaN,hampden,hampden,Unlawful sexual contact,Sexual Assault
51680,2020129743,2020129743360101,3601,1,NaN,NaN,323.0,1,0,1,2020-02-28T16:00:00,2020-02-28T20:13:00,NaN,hampden,hampden,Fondling of an adult,Sexual Assault
51681,2018553175,2018553175360101,3601,1,NaN,NaN,511.0,1,0,1,2018-08-14T11:35:00,2018-08-14T12:08:00,NaN,central-park,central-park,Fondling of an adult,Sexual Assault
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374256,20205002865,20205002865119901,1199,1,NaN,NaN,423.0,1,0,1,2020-08-04T06:35:00,2020-08-04T06:35:00,NaN,marston,marston,Sexual assault w/object by a person in a posit...,Sexual Assault
376629,2022330990,2022330990360101,3601,1,NaN,NaN,311.0,1,0,1,2022-06-27T18:00:00,2022-06-30T17:14:00,NaN,speer,speer,Fondling of an adult,Sexual Assault
376646,2022329687,2022329687110200,1102,0,NaN,NaN,222.0,1,0,1,2022-05-20T00:00:00,2022-06-29T19:39:00,NaN,hale,hale,Rape,Sexual Assault
376647,2022330901,2022330901110200,1102,0,NaN,NaN,423.0,1,0,1,2022-06-26T20:00:00,2022-06-30T16:20:00,NaN,marston,marston,Rape,Sexual Assault


In [45]:
# check the percentage of na
(len(rows_w_na) / len(dv_crime_data)) * 100

1.5522329437491373

In [46]:
SA = rows_w_na["offense_category_name"] == "Sexual Assault"

In [47]:
len(SA)

5849

In [48]:
len(rows_w_na)

5849

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Refering to the code above we see that around 1.5% of the data has NA values. Looking at the table these NA values are on the column geo_lon, geo_lat, and incident_street. This makes sense as geo_lon, geo_lat is dependent on incident_street. As the amount of NA is so low and all of the crimes are sexula assualt these we have decided to keep this data. Sexual assault is historically under reported therefore we think it important to leave this data in so we can represent the crime as well as we can. Moreover it is likely because of the nature of the crime and protection of the victim that the street address may not be reported for these crimes. 


## Step 8 
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Next the clean data is saved to a csv. With the clean data the next steps includes filtering and plotting the data for analysis. 

In [49]:
# save clean data to csv
dv_crime_data.to_csv('dv_crime_data.csv', index=False)

# Analysis  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Now using the clean data we filter graph our data for analysis. To filter by year the date is converted to a datetime object from ISO, however we decided to stick to ISO format as it is more standard and is easily converted to other types if needed as seen below.  For the first group of tables we split the large table by year based on the first_occurrence_date.

In [50]:
dv_crime_data_2017 = dv_crime_data[pd.to_datetime(dv_crime_data['first_occurrence_date']).dt.year == 2017]

In [51]:
dv_crime_data_2018 = dv_crime_data[pd.to_datetime(dv_crime_data['first_occurrence_date']).dt.year == 2018]

In [52]:
dv_crime_data_2019 = dv_crime_data[pd.to_datetime(dv_crime_data['first_occurrence_date']).dt.year == 2019]

In [53]:
dv_crime_data_2020 = dv_crime_data[pd.to_datetime(dv_crime_data['first_occurrence_date']).dt.year == 2020]

In [54]:
dv_crime_data_2021 = dv_crime_data[pd.to_datetime(dv_crime_data['first_occurrence_date']).dt.year == 2021]

In [55]:
dv_crime_data_2022 = dv_crime_data[pd.to_datetime(dv_crime_data['first_occurrence_date']).dt.year == 2022]

## Geolocation plots by year 
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; With these tables we create the first group of plots using kepler gl. Using the long and lat columns we plot the geo location for the crimes split by year. 

kepler docs: https://docs.kepler.gl/docs/keplergl-jupyter#keplergl

In [112]:
# imports for graphs 
# !pip install keplergl
# !jupyter labextension install @jupyter-widgets/jupyterlab-manager keplergl-jupyter
# !jupyter nbextension enable --py widgetsnbextension
# from keplergl import KeplerGl

In [109]:
dv_crime_data_2017_map = KeplerGl()
dv_crime_data_2017_map.add_data(data=dv_crime_data_2017, name='DV CRIME 2017')
dv_crime_data_2017_map

KeplerGl(data={'DV CRIME': {'index': [690, 698, 700, 702, 710, 711, 712, 716, 724, 729, 731, 753, 759, 766, 77…

In [110]:
dv_crime_data_2018_map = KeplerGl()
dv_crime_data_2018_map.add_data(data=dv_crime_data_2018, name='DV CRIME 2018')
dv_crime_data_2018_map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'DV CRIME 2018': {'index': [670, 671, 674, 686, 689, 691, 692, 693, 695, 708, 720, 727, 728, 73…

In [ ]:
dv_crime_data_2019_map = KeplerGl()
dv_crime_data_2019_map.add_data(data=dv_crime_data_2019, name='DV CRIME 2019')
dv_crime_data_2019_map

In [ ]:
dv_crime_data_2020_map = KeplerGl()
dv_crime_data_2020_map.add_data(data=dv_crime_data_2020, name='DV CRIME 2020')
dv_crime_data_2020_map

In [ ]:
dv_crime_data_2021_map = KeplerGl()
dv_crime_data_2021_map.add_data(data=dv_crime_data_2021, name='DV CRIME 2021')
dv_crime_data_2021_map

In [ ]:
dv_crime_data_2022_map = KeplerGl()
dv_crime_data_2022_map.add_data(data=dv_crime_data_2020, name='DV CRIME 2022')
dv_crime_data_2022_map

## Frequency graphs for crimes per year  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 

## Frequency graphs for crimes per month

## Frequency graphs for crimes before and after covid